In [2]:
import pandas as pd
from pathlib import pd

In [9]:
cases_train = pd.read_csv('./project_data/cases_2021_train_processed_2.csv')
cases_test = pd.read_csv('./project_data/cases_2021_test_processed_unlabelled_2.csv')

In [11]:
cases_train

,age,sex,province,country,latitude,longitude,date_confirmation,chronic_disease_binary,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,outcome_group
0,18,female,Bihar,India,25.49096,85.93903,2020-05-18,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
1,27,female,NaN,Philippines,7.07000,125.60000,2020-04-15,False,747288,13297,603746,130245,681.949809,1.779368,nonhospitalized
2,46,male,Tamil Nadu,India,13.08362,80.28252,2020-05-02,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
3,21,female,Tamil Nadu,India,13.08362,80.28252,2020-05-24,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
4,27,male,Bihar,India,26.28361,87.20347,2020-05-27,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17207,29,female,Tamil Nadu,India,13.08362,80.28252,2020-04-24,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
17208,47,female,NaN,Philippines,16.68814,121.54992,2020-04-09,False,747288,13297,603746,130245,681.949809,1.779368,nonhospitalized
17209,30,male,Bihar,India,24.94452,86.26404,2020-05-26,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
17210,59,male,Tamil Nadu,India,12.68224,79.98008,2020-05-31,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized


In [12]:
cases_train.drop(columns=['province', 'country', 'date_confirmation'], inplace=True)